In [36]:
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    PER,
    NamesExtractor,
    Doc
)
from rdflib import Graph, Namespace, URIRef

In [37]:
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

In [38]:
text_IT = open(r'C:\Users\Юлия\OneDrive\Документы\Универ\3 курс\8 триместр\Курсовая\Semantic annotation\files\text_IT.txt', encoding='UTF-8').read()
text_Friends = open(r'C:\Users\Юлия\OneDrive\Документы\Универ\3 курс\8 триместр\Курсовая\Semantic annotation\files\text_Friends.txt', encoding='UTF-8').read()

# print(f'Текст IT:\n{text_IT}\n\nТекст Fiends:\n{text_Friends}')

In [39]:
ontology_graph = Graph()
ontology_graph.parse(r'C:\Users\Юлия\OneDrive\Документы\Универ\3 курс\8 триместр\Курсовая\Semantic annotation\files\ontologies.owl')
# ontology_graph.print()

<Graph identifier=Na20aebb8739644f29d4afb97867587cc (<class 'rdflib.graph.Graph'>)>

In [40]:
doc_IT = Doc(text_IT)
doc_Friends = Doc(text_Friends)

In [41]:
doc_IT.segment(segmenter)
doc_Friends.segment(segmenter)

In [42]:
doc_IT.tag_morph(morph_tagger)
doc_Friends.tag_morph(morph_tagger)

In [43]:
doc_IT.parse_syntax(syntax_parser)
doc_Friends.parse_syntax(syntax_parser)